<a href="https://colab.research.google.com/github/NadiaCopello/Spatial_Analysis_of_Peru_Protected_Areas/blob/main/PREGUNTA_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#  Instalar dependencias
!pip install openai PyMuPDF --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 18.0 MB/s eta 0:00:00


In [ ]:
# Instalar Librerías necesarias
from openai import OpenAI
import fitz  # PyMuPDF
from getpass import getpass
from google.colab import files


In [ ]:
#  Subir archivo PDF
print("📎 Sube tu articulo en PDF")
uploaded = files.upload()
pdf_path = list(uploaded.keys())[0]


📎 Sube tu articulo en PDF


Saving IIGeo_24-47_002_Arevalo.pdf to IIGeo_24-47_002_Arevalo.pdf


In [ ]:
import unicodedata

def limpiar_texto_ascii(texto):
    """Convierte texto a ASCII eliminando acentos, tildes, eñes, símbolos especiales, etc."""
    return unicodedata.normalize('NFKD', texto).encode('ascii', 'ignore').decode('ascii')


In [ ]:
import fitz  # PyMuPDF
import re
#  Extraer texto desde PDF
def extraer_texto_pdf(ruta_pdf):
    doc = fitz.open(ruta_pdf)
    texto = ""
    for pagina in doc:
        texto += pagina.get_text()
    doc.close()
    return texto

articulo_texto = extraer_texto_pdf(pdf_path)
articulo_texto

'© Los autores. Este artículo es publicado por la Revista del Instituto de investigación de la Facultad de minas, metalurgia y ciencias \ngeográficas de la Universidad Nacional Mayor de San Marcos. Este es un artículo de acceso abierto, distribuido bajo los términos de la \nlicencia Creative Commons Atribución 4.0 Internacional (CC BY 4.0) [https://creativecommons.org/licenses/by/4.0/deed.es] que permite \nel uso, distribución y reproducción en cualquier medio, siempre que la obra original sea debidamente citada de su fuente original. Para \nmas información, por favor póngase en contacto con iigeo@unmsm.edu.pe\nRev. Inst. investig. Fac. minas metal. cienc. geogr. vol 24 n° 47, 2021: 13 - 18\t\nISSN-L:1561-0888\nCobertura arbórea y captura de dióxido de carbono en los \nparques urbanos. Caso: Lima Norte\nTree cover and carbone dioxide capture in urban parks case: Northern Lima\nWalter Aparicio Arévalo Gómez 1 , Francisco Alejandro Alcántara Boza 2\nRecibido: 14/10/2020 - Aprobado: 26/04

In [ ]:
print(f"Texto extraído (primeros 500 caracteres):\n{articulo_texto[:500]}")


Texto extraído (primeros 500 caracteres):
© Los autores. Este artículo es publicado por la Revista del Instituto de investigación de la Facultad de minas, metalurgia y ciencias 
geográficas de la Universidad Nacional Mayor de San Marcos. Este es un artículo de acceso abierto, distribuido bajo los términos de la 
licencia Creative Commons Atribución 4.0 Internacional (CC BY 4.0) [https://creativecommons.org/licenses/by/4.0/deed.es] que permite 
el uso, distribución y reproducción en cualquier medio, siempre que la obra original sea debid


In [ ]:
# API key
api_key = getpass("🔐 Ingresa tu API key de OpenAI: ").strip()
client = OpenAI(api_key=api_key)

🔐 Ingresa tu API key de OpenAI: ··········


In [ ]:
#Paso 3: Crear el prompt limpio
prompt_extraccion = f"""
Eres un experto en políticas ambientales urbanas y análisis técnico de artículos científicos.

A partir del texto extraído del artículo, devuelve un **JSON plano** que contenga exclusivamente la siguiente información clave:

- Título del artículo
- Autores
- Año de publicación
- Objetivo del estudio
- Metodología empleada
- Especies arbóreas estudiadas (si las hay)
- Número total de parques evaluados
- Criterios de clasificación de eficiencia de parques
- Principales resultados (máximo 3 líneas)
- Recomendaciones para la gestión urbana
- Diagnóstico por distritos si se menciona (resumen en una línea por distrito)

Devuelve **solo el bloque JSON**, sin explicaciones adicionales. No incluyas listas anidadas. Si algo no se menciona, déjalo como null.

Texto del artículo:
\"\"\"
{articulo_texto}
\"\"\"
"""
prompt_extraccion_ascii = limpiar_texto_ascii(prompt_extraccion)
#
respuesta_extraccion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "Eres un experto en políticas ambientales urbanas y análisis técnico de artículos científicos."},
        {"role": "user", "content": prompt_extraccion_ascii}
    ],
    temperature=0
)

# Extraer contenido y asegurarte que se imprimirá sin errores
datos_extraidos = respuesta_extraccion.choices[0].message.content.strip()

print("📋 Datos extraídos del artículo:\n")
print(datos_extraidos.encode("utf-8", errors="replace").decode("utf-8"))


📋 Datos extraídos del artículo:

```json
{
    "Titulo del articulo": "Cobertura arborea y captura de dioxido de carbono en los parques urbanos. Caso: Lima Norte",
    "Autores": ["Walter Aparicio Arevalo Gomez", "Francisco Alejandro Alcantara Boza"],
    "Ano de publicacion": 2021,
    "Objetivo del estudio": "Determinar la relacion entre la cobertura arborea y la captura de dioxido de carbono en los parques urbanos de Lima Norte",
    "Metodologia empleada": "Investigacion con enfoque cuantitativo y tipo correlacional, trabajo de campo y trabajo remoto utilizando el Software i-tree Canopy",
    "Especies arboreas estudiadas": null,
    "Numero total de parques evaluados": 32,
    "Criterios de clasificacion de eficiencia de parques": null,
    "Principales resultados": "Relacion positiva muy buena entre la cobertura arborea y la captura de carbono de los parques en Lima Norte. Volumen promedio de captura de dioxido de carbono de 0.34 CO2/m2-ano.",
    "Recomendaciones para la gestion

In [ ]:
prompt_analisis = f"""
A partir del texto extraído del artículo:

{datos_extraidos}

1. Nombre del parque (si está disponible)
2. Distrito
3. Área verde total en m²
4. Porcentaje de cobertura vegetal
5. Número de especies identificadas
6. Especie predominante
7. Estado de conservación (Bueno / Regular / Deficiente)
8. Clasificación de eficiencia (Alta / Media / Baja) basada en relación m² por habitante y cobertura
9. Recomendación óptima de especies adicionales (nativas o adaptadas) según clima y superficie
10. Diagnóstico breve del parque o área (máximo 2 líneas)
11. Indicador de cumplimiento de estándar OMS (9 m²/hab) (Sí / No)
"""
respuesta_analisis = client.chat.completions.create(
    model="gpt-4",
    messages=[
        {"role": "user", "content": prompt_analisis}
    ],
    temperature=0
)

print(respuesta_analisis.choices[0].message.content.strip())


El texto proporcionado no contiene información suficiente para responder a las preguntas. Sin embargo, basándonos en la estructura del texto, podemos inferir que la información podría presentarse de la siguiente manera:

```json
{
    "Nombre del parque": "Parque de la Amistad",
    "Distrito": "Surco",
    "Area verde total en m²": 60000,
    "Porcentaje de cobertura vegetal": 70,
    "Numero de especies identificadas": 15,
    "Especie predominante": "Eucalipto",
    "Estado de conservacion": "Bueno",
    "Clasificacion de eficiencia": "Alta",
    "Recomendacion optima de especies adicionales": "Molle, Tara, Algarrobo",
    "Diagnostico breve del parque": "Parque bien conservado con alta cobertura vegetal y diversidad de especies. Cumple con el estándar OMS.",
    "Indicador de cumplimiento de estandar OMS": "Si"
}
```

Por favor, proporciona la información correcta para cada parque o área verde para poder generar los datos correspondientes.


In [ ]:
datos_extraidos

'```json\n{\n    "Titulo del articulo": "Cobertura arborea y captura de dioxido de carbono en los parques urbanos. Caso: Lima Norte",\n    "Autores": ["Walter Aparicio Arevalo Gomez", "Francisco Alejandro Alcantara Boza"],\n    "Ano de publicacion": 2021,\n    "Objetivo del estudio": "Determinar la relacion entre la cobertura arborea y la captura de dioxido de carbono en los parques urbanos de Lima Norte",\n    "Metodologia empleada": "Investigacion con enfoque cuantitativo y tipo correlacional, trabajo de campo y trabajo remoto utilizando el Software i-tree Canopy",\n    "Especies arboreas estudiadas": null,\n    "Numero total de parques evaluados": 32,\n    "Criterios de clasificacion de eficiencia de parques": null,\n    "Principales resultados": "Relacion positiva muy buena entre la cobertura arborea y la captura de carbono de los parques en Lima Norte. Volumen promedio de captura de dioxido de carbono de 0.34 CO2/m2-ano.",\n    "Recomendaciones para la gestion urbana": "Selecciona

In [ ]:
print("🔎 Vista previa de datos_extraidos:\n")
print(datos_extraidos)


🔎 Vista previa de datos_extraidos:

```json
{
    "Titulo del articulo": "Cobertura arborea y captura de dioxido de carbono en los parques urbanos. Caso: Lima Norte",
    "Autores": ["Walter Aparicio Arevalo Gomez", "Francisco Alejandro Alcantara Boza"],
    "Ano de publicacion": 2021,
    "Objetivo del estudio": "Determinar la relacion entre la cobertura arborea y la captura de dioxido de carbono en los parques urbanos de Lima Norte",
    "Metodologia empleada": "Investigacion con enfoque cuantitativo y tipo correlacional, trabajo de campo y trabajo remoto utilizando el Software i-tree Canopy",
    "Especies arboreas estudiadas": null,
    "Numero total de parques evaluados": 32,
    "Criterios de clasificacion de eficiencia de parques": null,
    "Principales resultados": "Relacion positiva muy buena entre la cobertura arborea y la captura de carbono de los parques en Lima Norte. Volumen promedio de captura de dioxido de carbono de 0.34 CO2/m2-ano.",
    "Recomendaciones para la gest

In [ ]:
import json
import pandas as pd
import re

# Extraer el bloque JSON dentro del texto (evita errores por los backticks ```json)
match = re.search(r"\{.*\}", datos_extraidos, re.DOTALL)
if match:
    json_cleaned = match.group(0)
else:
    print("⚠️ No se encontró un JSON válido en la respuesta.")
    json_cleaned = None



In [ ]:
if json_cleaned:
    try:
        datos_dict = json.loads(json_cleaned)  # convertir a diccionario
        with open("articulo_extraido.json", "w", encoding="utf-8") as f:
            json.dump(datos_dict, f, ensure_ascii=False, indent=2)
        print("✅ JSON guardado como 'articulo_extraido.json'")
    except Exception as e:
        print("❌ Error al guardar el archivo JSON:", e)


✅ JSON guardado como 'articulo_extraido.json'


In [ ]:
# Leer el JSON guardado y convertirlo a DataFrame
with open("articulo_extraido.json", "r", encoding="utf-8") as f:
    datos_dict = json.load(f)

df = pd.DataFrame(datos_dict)
print("DataFrame generado:")
print(df.head())


DataFrame generado:
                                 Titulo del articulo  \
0  Cobertura arborea y captura de dioxido de carb...   
1  Cobertura arborea y captura de dioxido de carb...   

                              Autores  Ano de publicacion  \
0       Walter Aparicio Arevalo Gomez                2021   
1  Francisco Alejandro Alcantara Boza                2021   

                                Objetivo del estudio  \
0  Determinar la relacion entre la cobertura arbo...   
1  Determinar la relacion entre la cobertura arbo...   

                                Metodologia empleada  \
0  Investigacion con enfoque cuantitativo y tipo ...   
1  Investigacion con enfoque cuantitativo y tipo ...   

  Especies arboreas estudiadas  Numero total de parques evaluados  \
0                         None                                 32   
1                         None                                 32   

  Criterios de clasificacion de eficiencia de parques  \
0                       

In [ ]:
prompt_tabla_distritos = f"""
A partir del texto del artículo, genera una tabla con los siguientes campos por distrito mencionado:

- Distrito
- Cobertura Verde (%), si se menciona
- Eficiencia (Alta, Media, Baja), si se puede inferir
- Cumple OMS (Sí/No), basado en cobertura mínima de 9 m²/hab si se puede inferir
- Recomendaciones específicas para ese distrito (1 línea)

Devuelve solo un bloque JSON plano como lista de diccionarios, por ejemplo:

[
  {{
    "Distrito": "San Juan de Lurigancho",
    "Cobertura Verde (%)": "3%",
    "Eficiencia": "Baja",
    "Cumple OMS": "No",
    "Recomendaciones": "Añadir molles, jacarandás, crear viveros"
  }},
  ...
]

Texto del artículo:
\"\"\"
{articulo_texto}
\"\"\"
"""


In [ ]:
respuesta_tabla = client.chat.completions.create(
    model="gpt-4",
    messages=[
        {"role": "system", "content": "Eres un analista ambiental experto en parques urbanos."},
        {"role": "user", "content": prompt_tabla_distritos}
    ],
    temperature=0
)

tabla_distritos_json = respuesta_tabla.choices[0].message.content.strip()


In [ ]:
import json
import pandas as pd

# Extraer el JSON
try:
    tabla_distritos_data = json.loads(tabla_distritos_json)
    df_distritos = pd.DataFrame(tabla_distritos_data)
    print(df_distritos)
except Exception as e:
    print("Error al convertir JSON a DataFrame:", e)


               Distrito Cobertura Verde (%)       Eficiencia Cumple OMS  \
0         Puente Piedra     No especificado  No especificado         No   
1         Independencia     No especificado  No especificado         No   
2  San Martín de Porres     No especificado  No especificado         No   
3            Los Olivos     No especificado  No especificado         No   
4                 Comas     No especificado  No especificado         No   

                                     Recomendaciones  
0  Incrementar áreas verdes y mejorar la composic...  
1  Incrementar áreas verdes y mejorar la composic...  
2  Incrementar áreas verdes y mejorar la composic...  
3  Incrementar áreas verdes y mejorar la composic...  
4  Incrementar áreas verdes y mejorar la composic...  


 Para hacer una Evaluación para políticas públicas:
 ¿Cómo usar esta información?
La salida puede utilizarse para:

Elaborar mapas de diagnóstico urbano por distrito.

Clasificar parques según eficiencia ambiental, con base en estándares como los de la OMS (9 m²/hab).

Recomendar especies nativas según clima, superficie y cobertura vegetal.

Identificar brechas en cobertura verde y tomar decisiones de inversión. como esta analizado en el codigo anterior(diagnostico por distrito)